In [ ]:
#
# Stub Bundling and Confluent Spirals for Geographic Networks
# - Arlind Nocaj and Ulrik Brandes
# - Department of Computer & Information Science, University of Konstanz
# S. Wismath and A. Wolff (Eds.): GD 2013, LNCS 8242, pp. 388–399, 2013.
#
from math import pi, cos, sin, tan, atan2, exp, sqrt, copysign, acos
import rtsvg
rt = rtsvg.RACETrack()

import random
def randomPoints(n_min=2, n_max=10, d=8.0, o=1.0):
    v  = (5.0 - 10.0*random.random(), 5.0 - 10.0*random.random())
    a  = random.random() * 2 * pi
    n  = random.randint(n_min, n_max)
    ws = []
    for i in range(n):
        d0            = 3.0 + random.random()*d
        u_vec, v_vec  = cos(a), sin(a)
        x0,y0         = v[0] + u_vec*d0, v[1] + v_vec*d0
        o0            = random.gauss(sigma=o)
        ws.append((x0+v_vec*o0, y0-u_vec*o0))
    return v, ws

#_tiles_ = []
#for i in range(8):
#    v,ws = randomPoints()
#    _tiles_.append(ConfluentSpiralTree(v,ws))
#rt.table(_tiles_, per_row=4, spacer=5)

class SVGGridPaper(object):
    #
    # __init__() - constructor
    # ... creates grid paper and fills it in with the points objects
    # ... those objects should have a member variable called "pts" which is a list of (x,y) tuples
    #
    def __init__(self, labeled_points=None, w=512, h=512, draw_segment_points=True, draw_labels=True):
        self.w, self.h           = w, h
        self.draw_segment_points = draw_segment_points
        self.draw_labels         = draw_labels
        self.labeled_points      = labeled_points
        self.pts_objs            = []

    #
    # add() - add more points objects
    #
    def add(self, *pts):
        self.pts_objs.extend(pts)

    #
    # SVG Representation
    #
    def _repr_svg_(self):
        x0, y0, x1, y1 = 0.0, 0.0, 1.0, 1.0
        for k in self.labeled_points:
            _xy_ = self.labeled_points[k]
            x0, y0, x1, y1 = min(x0, _xy_[0]), min(y0, _xy_[1]), max(x1, _xy_[0]), max(y1, _xy_[1])
        if len(self.pts_objs) > 0:
            for _pts_ in self.pts_objs:
                for _pt_ in _pts_.pts: x0, y0, x1, y1 = min(x0,_pt_[0]), min(y0,_pt_[1]), max(x1,_pt_[0]), max(y1,_pt_[1])            
        x_perc = 0.05 * (x1 - x0)
        y_perc = 0.05 * (y1 - y0)
        x0, y0, x1, y1 = x0 - x_perc, y0 - y_perc, x1 + x_perc, y1 + y_perc
        svg = [f'<svg x="0" y="0" width="{self.w}" height="{self.h}" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
        x = int(x0)
        if int(x1) - int(x0) < 200:
            while x < x1:
                svg.append(f'<line x1="{x}" y1="{y0}" x2="{x}" y2="{y1}" stroke="#a0a0a0" stroke-width="0.05"/>')
                x += 1
        y = int(y0)
        if int(y1) - int(y0) < 200:
            while y < y1:
                svg.append(f'<line x1="{x0}" y1="{y}" x2="{x1}" y2="{y}" stroke="#a0a0a0" stroke-width="0.05"/>')
                y += 1
        svg.append(f'<line x1="{0.0}" y1="{y0}"  x2="{0.0}" y2="{y1}"  stroke="#f04040" stroke-width="0.1"/>')
        svg.append(f'<line x1="{x0}"  y1="{0.0}" x2="{x1}"  y2="{0.0}" stroke="#f04040" stroke-width="0.1"/>')

        if self.labeled_points is not None:
            for _label_ in self.labeled_points:
                _xy_ = self.labeled_points[_label_]
                if self.draw_labels: svg.append(rt.svgText(_label_, _xy_[0], _xy_[1]-0.5, txt_h=1.5, color='#a0a0a0', anchor='middle'))
                svg.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="0.2" fill="#ff0000"/>')

        for _pts_ in self.pts_objs:
            if _pts_.color is not None: _color_ = _pts_.color
            else:                       _color_ = '#000000'
            _path_ = [f'M {_pts_.pts[0][0]} {_pts_.pts[0][1]}']
            for i in range(1, len(_pts_.pts)): _path_.append(f'L {_pts_.pts[i][0]} {_pts_.pts[i][1]}')
            svg.append(f'<path d="{" ".join(_path_)}" fill="none" stroke="{_color_}" stroke-width="0.1"/>')
            if self.draw_segment_points:
                for i in range(len(_pts_.pts)):
                    _pt_ = _pts_.pts[i]
                    svg.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="0.1" fill="{_color_}"/>')
        svg.append('</svg>')
        return ''.join(svg)

class GridPaperLine(object):
    def __init__(self, xy0, xy1, color='#c0c0c0'): self.pts, self.color = [xy0,xy1], color


In [ ]:
#
# LogarithmicSpiral class
# - Section 4.1 of the paper
# - Created from Gemini 2.5 Pro
#
class LogarithmicSpiral(object):
    #
    # Constructor
    #
    def __init__(self, pv, pw, theta=pi/4.0, t_inc=0.1, t_min=0.0, t_max=4.0):
        self.pv,    self.pw,    self.theta = pv,    pw,    theta
        self.t_inc, self.t_min, self.t_max = t_inc, t_min, t_max
        self._length_                      = None
        self.color = '#000000'

        b = 1.0 / tan(-theta) if theta < 0.0 else 1.0 / tan( theta)
        u_wv_x = pv[0] - pw[0]
        u_wv_y = pv[1] - pw[1]
        L0     = sqrt(u_wv_x**2 + u_wv_y**2)
        phi0   = atan2(u_wv_y, u_wv_x)

        self.pts = []
        t        = t_min
        while t <= t_max:
            radial_factor = L0 * exp(-b * t)
            # Angle depends on the sign of b according to the paper's convention
            # Angle = phi0 + sgn(b) * t
            sign_theta    = copysign(1.0, theta) # Get sign of theta (+1.0 or -1.0)
            current_angle = phi0 + sign_theta * t
            # Calculate displacement vector components
            delta_x, delta_y = radial_factor * cos(current_angle), radial_factor * sin(current_angle)
            # Calculate the absolute coordinates by adding displacement to the pole
            x, y = pw[0] + delta_x, pw[1] + delta_y
            self.pts.append((x,y))
            t += t_inc
    def length(self):
        if self._length_ is None:
            self._length_ = 0.0
            for i in range(1, len(self.pts)): self._length_ += sqrt((self.pts[i][0] - self.pts[i-1][0])**2 + (self.pts[i][1] - self.pts[i-1][1])**2)
        return self._length_
    def _repr_svg_(self):
        svg_grid_paper = SVGGridPaper(labeled_points={'pv':self.pv, 'pw':self.pw}, w=256, h=256)
        svg_grid_paper.add(self)
        return svg_grid_paper._repr_svg_()


_tiles_    = []
_pv_, _pw_ = (1.0, 2.0),(-17.0, 9.0)
for _pos_ in [1.0, -1.0]:
    svg_grid_paper = SVGGridPaper(labeled_points={'pv':_pv_, 'pw':_pw_}, w=256, h=256)
    for _div_ in [3.0, 5.0, 9.0, 21.0]: 
        _log_spiral_ = LogarithmicSpiral(_pv_, _pw_, theta=_pos_ * pi/_div_)
        svg_grid_paper.add(_log_spiral_)
    _tiles_.append(svg_grid_paper)
for i in range(5):
    _tiles_.append(LogarithmicSpiral((-10 + 20.0*random.random(), -10 + 20.0*random.random()), (-10 + 20.0*random.random(), -10 + 20.0*random.random())))
for i in range(5):
    _tiles_.append(LogarithmicSpiral((-10 + 20.0*random.random(), -10 + 20.0*random.random()), (-10 + 20.0*random.random(), -10 + 20.0*random.random())))
rt.table(_tiles_, spacer=10, per_row=6)

In [ ]:
#
# Section 4.2 of the paper
# - Assumes that the points are bundled in the same / similar direction
# - manual version of the algorithm to see how it works
#
v, w1, w2, w3 = (1.0, 3.0), (15.0, 6.0), (5.0, 5.0), (12.0, 8.0)
ws = [w1, w2, w3]
def sortPointsByDistance(v, ws): return sorted(ws, key=lambda w: rt.segmentLength((v, w)))
ws = sortPointsByDistance(v, ws)

svg_grid_paper = SVGGridPaper(labeled_points={'v':v, 'w1':w1, 'w2':w2, 'w3':w3}, draw_segment_points=False, draw_labels=False)

_angle_ave_ = 0.0
for _w_ in ws: _angle_ave_ += atan2(_w_[1]-v[1], _w_[0]-v[0])
_angle_ave_ /= len(ws)

_trunk_ = LogarithmicSpiral(v, ws[0], theta=_angle_ave_)
svg_grid_paper.add(_trunk_)

i         = 2
_angle_   = atan2(_trunk_.pts[i][1] - _trunk_.pts[i-1][1], _trunk_.pts[i][0] - _trunk_.pts[i-1][0])
_branch1_ = LogarithmicSpiral(_trunk_.pts[i], ws[1], theta=_angle_)
svg_grid_paper.add(_branch1_)

i         = 3
_angle_   = atan2(_branch1_.pts[i][1] - _branch1_.pts[i-1][1], _branch1_.pts[i][0] - _branch1_.pts[i-1][0])
_branch2_ = LogarithmicSpiral(_branch1_.pts[i], ws[2], theta=-_angle_)
svg_grid_paper.add(_branch2_)

svg_grid_paper

In [ ]:
#
# Confluent Spiral Trees
# - Automated version of the last cell
# - Section 4.2 of the paper
#
class ConfluentSpiralTree(object):
    def _repr_svg_(self):
        _labels_ = {'v':self.v}
        for i in range(len(self.ws)): _labels_[f'w{i+1}'] = self.ws[i]
        svg_grid_paper = SVGGridPaper(labeled_points=_labels_, draw_segment_points=False, draw_labels=False, w=384, h=384)
        svg_grid_paper.add(GridPaperLine(self.v, (self.v[0] + 5.0*self.u_avg, self.v[1] + 5.0*self.v_avg)))
        for _spiral_ in self.spirals: svg_grid_paper.add(_spiral_)
        return svg_grid_paper._repr_svg_()

    def __init__(self, v, ws):
        # Helper functions
        def unitVector(_xvec_, _yvec_):               # unit vector
            _l_ = sqrt(_xvec_**2 + _yvec_**2)
            if _l_ < 10e-6: _l_ = 0.01
            return _xvec_/_l_, _yvec_/_l_
        cp = lambda v0, v1: v0[0]*v1[1] - v0[1]*v1[0] # cross product
        dp = lambda v0, v1: v0[0]*v1[0] + v0[1]*v1[1] # dot product
        # Sort ws by distance -- place the nearest point first...
        ws = sorted(ws, key=lambda w: sqrt((v[1]-w[1])**2 + (v[0]-w[0])**2))
        self.v, self.ws, self.spirals, u_sum, v_sum = v, ws, [], 0.0, 0.0
        # Calculate the average vector to all of the points (ws)
        for w in self.ws: 
            _u_,    _v_   = unitVector(w[0]-v[0], w[1]-v[1])
            u_sum,  v_sum = u_sum + _u_, v_sum + _v_
        self.u_avg, self.v_avg = unitVector(u_sum/len(self.ws), v_sum/len(self.ws))
        # Create the trunk of the tree - the very first log spiral
        self.theta_avg = atan2(self.v_avg, self.u_avg)
        if abs(self.theta_avg) > pi/5: self.theta_avg = pi/5
        _u_, _v_ = unitVector(self.ws[0][0] - self.v[0], self.ws[0][1] - self.v[1])
        _sign_ = 1.0 if cp((self.u_avg,self.v_avg),(_u_,_v_)) > 0.0 else -1.0
        self.spirals.append(LogarithmicSpiral(self.v, self.ws[0], theta=_sign_*self.theta_avg)) # acos(dp((u,v),(self.u_avg, self.v_avg)))))
        # For all of the other ws points, connect them to the tree
        # ... The paper recommends a sampled version...
        # ... However, it seems that looking at the first several points of each spiral produces good results
        deferred_ws_indices, self.not_placed = [], []
        for _iteration_ in range(2):
            if _iteration_ == 0: _indices_to_calculate_, _min_points_, _max_pi_ = range(1, len(self.ws)),    5,  pi/8
            else:                _indices_to_calculate_, _min_points_, _max_pi_ = deferred_ws_indices,      10,  pi/3
            for i in _indices_to_calculate_:
                best_log_spiral = None # capture the best spiral found so far...
                for spiral_i in range(len(self.spirals)-1, -1, -1):
                    _spiral_ = self.spirals[spiral_i]                                                                   # Checking this spiral for a best fit
                    for _pts_i_ in range(1, min(_min_points_, len(_spiral_.pts))):                                      # Go through earlier points for best fit
                        _u_seg_, _v_seg_ = unitVector(_spiral_.pts[_pts_i_][0] - _spiral_.pts[_pts_i_-1][0], 
                                                      _spiral_.pts[_pts_i_][1] - _spiral_.pts[_pts_i_-1][1])            # Spiral Segment Vector
                        _u_pw_, _v_pw_   = unitVector(self.ws[i][0]  - _spiral_.pts[_pts_i_][0], 
                                                      self.ws[i][1]  - _spiral_.pts[_pts_i_][1])                        # Vector to the spiral-to-place
                        _angle_to_pw_    = acos(dp((_u_pw_, _v_pw_)  ,(_u_seg_, _v_seg_)))                              # As an angle (from the segment uv)
                        if abs(_angle_to_pw_) < 0.2 or abs(_angle_to_pw_) > _max_pi_: continue                          # Constraints (not exactly like paper)
                        _sign_ = 1.0 if cp((_u_seg_,_v_seg_),(_u_pw_,_v_pw_)) > 0.0 else -1.0                           # Which side of the line?
                        _log_spiral_ = LogarithmicSpiral(_spiral_.pts[_pts_i_], self.ws[i], theta=_sign_*_angle_to_pw_) # Create the spiral
                        if   best_log_spiral is None:                           best_log_spiral = _log_spiral_          # Keep if not already set
                        elif best_log_spiral.length() > _log_spiral_.length():  best_log_spiral = _log_spiral_          # keep if shorter than best found so far
                if   best_log_spiral is not None: self.spirals.append(best_log_spiral)                                  # Append the best ...
                elif _iteration_ == 0:            deferred_ws_indices.append(i)                                         # ... or defer to the second iteration
                elif _iteration_ == 1:            self.not_placed.append(i)                                             # ... or report out that the points were never placed

_tiles_ = []
_tiles_.append(ConfluentSpiralTree((1.0, 3.0), [(15.0, 6.0), ( 5.0, 5.0), (12.0, 8.0)]))
_tiles_.append(ConfluentSpiralTree((1.0, 3.0), [(15.0, 6.0), ( 5.0, 3.0), (12.0, 8.0)]))
_tiles_.append(ConfluentSpiralTree((1.0, 3.0), [(15.0, 6.0), (14.0, 3.0), (12.0, 8.0)]))
_tiles_.append(ConfluentSpiralTree((-1.0, -3.0), [(-15.0, -6.0), ( -5.0, -5.0), (-12.0, -8.0)]))
_tiles_.append(ConfluentSpiralTree((3.31, 0.13), [(3.88, -3.15), (5.54, -6.05), (5.79, -6.43),   (6.97, -7.25)]))
_tiles_.append(ConfluentSpiralTree((3.31, 0.13), [(3.88, -3.15)]))
rt.table(_tiles_, per_row=4, spacer=5)

In [ ]:
_tiles_ = []
for i in range(32):
    v,ws = randomPoints()
    _tiles_.append(ConfluentSpiralTree(v,ws))
rt.table(_tiles_, per_row=4, spacer=5)